## IMPORT MODULES

In [1]:
from selection_service.enums.Enums import DesignCode, ProviderName
from selection_service.core.Pipeline import EarthquakeAPI
from selection_service.providers.Providers import ProviderFactory
from selection_service.processing.Selection import SelectionConfig,SearchCriteria,TBDYSelectionStrategy,TargetParameters
from selection_service.core.LoggingConfig import setup_logging
from selection_service.core.Pipeline import get_selected_earthquake
# import logging
setup_logging()

## PREPROCESSING

In [2]:
providerFactory = ProviderFactory()
afadProvider = providerFactory.create_provider(ProviderName.AFAD, station_file_path="D:\\github\\SelectionEarthquake\\data\\stations.xlsx")
peerProvider = providerFactory.create_provider(provider_type=ProviderName.PEER, file_path="D:\\github\\SelectionEarthquake\\data\\NGA-West2_flatfile.csv")
con = SelectionConfig(design_code=DesignCode.TBDY_2018, num_records=22, max_per_station=3, max_per_event=3, min_score=55)
strategy = TBDYSelectionStrategy(config=con)
search_criteria = SearchCriteria(start_date="2020-01-01", end_date="2025-09-05", min_magnitude=7.0, max_magnitude=10.0, min_vs30=400, max_vs30=500 )
target_params = TargetParameters(magnitude=7.0, distance=30.0, vs30=400.0, pga=300, mechanism=["StrikeSlip","Reverse"] )

## BASIC USAGE

### Asynchronously execute

In [3]:
result = await get_selected_earthquake(criteria=search_criteria, target=target_params, providers=[afadProvider,peerProvider], strategies=[strategy], async_mode=True)
result[['PROVIDER','RSN','EVENT','YEAR','MAGNITUDE','STATION','VS30(m/s)','RRUP(km)','MECHANISM','T90_avg(sec)','PGA(cm2/sec)','PGV(cm/sec)','SCORE']]

AFAD arama kriterleri: {'startDate': '2020-01-01T00:00:00.000Z', 'endDate': '2025-09-05T23:59:59.999Z', 'fromMagnitude': 7.0, 'toMagnitude': 10.0, 'fromVs30': 400, 'toVs30': 500}
AFAD'dan 114 kayıt alındı.


,PROVIDER,RSN,EVENT,YEAR,MAGNITUDE,STATION,VS30(m/s),RRUP(km),MECHANISM,T90_avg(sec),PGA(cm2/sec),PGV(cm/sec),SCORE
115,PEER,139,"Tabas, Iran",1978,7.35,Dayhook,471.53,13.940000,Reverse,0.000000,326.924291,25.872000,100.000000
112,AFAD,327918,17966,2023,7.70,"Necip Fazıl Kültür Merkezi, Alparslan Türkeş B...",484.00,37.225843,StrikeSlip,43.853333,320.930444,37.353823,91.304348
65,AFAD,327966,17966,2023,7.70,Ali Tekden Sağlık Ocağı,424.00,38.529217,StrikeSlip,31.463333,286.722564,48.332219,91.304348
64,AFAD,327952,17966,2023,7.70,Çok Programlı Anadolu Lisesi,448.00,20.737167,StrikeSlip,7.693333,291.290941,98.907393,91.304348
259,PEER,4451,"Montenegro, Yugo.",1979,7.10,Bar-Skupstina Opstine,462.23,6.980000,Reverse,0.000000,361.728092,49.588000,87.826087
366,PEER,8166,"Duzce, Turkey",1999,7.14,IRIGM 498,425.00,3.580000,StrikeSlip,0.000000,365.542879,25.269000,87.826087
392,PEER,8605,El Mayor-Cucapah,2010,7.20,Bird Spring,402.00,388.790000,StrikeSlip,0.000000,2.152265,2.253900,69.565217
260,PEER,4452,"Montenegro, Yugo.",1979,7.10,Debar - Skupstina Opstine,485.04,118.490000,Reverse,0.000000,55.347752,2.480000,69.565217
256,PEER,3821,Hector Mine,1999,7.13,San Pedro - Palos Verdes,477.92,206.310000,StrikeSlip,0.000000,8.361444,4.209900,69.565217
255,PEER,3798,Hector Mine,1999,7.13,Los Angeles - Colorado & Eagle Rock,418.83,177.390000,StrikeSlip,0.000000,20.845015,3.979400,69.565217


### Synchronously execute

In [4]:
result = await get_selected_earthquake(criteria=search_criteria, target=target_params, providers=[afadProvider,peerProvider], strategies=[strategy], async_mode=False)
result[['PROVIDER','RSN','EVENT','YEAR','MAGNITUDE','STATION','VS30(m/s)','RRUP(km)','MECHANISM','T90_avg(sec)','PGA(cm2/sec)','PGV(cm/sec)','SCORE']]

AFAD arama kriterleri: {'startDate': '2020-01-01T00:00:00.000Z', 'endDate': '2025-09-05T23:59:59.999Z', 'fromMagnitude': 7.0, 'toMagnitude': 10.0, 'fromVs30': 400, 'toVs30': 500}
AFAD'dan 114 kayıt alındı.
PEER'dan 279 kayıt alındı.


,PROVIDER,RSN,EVENT,YEAR,MAGNITUDE,STATION,VS30(m/s),RRUP(km),MECHANISM,T90_avg(sec),PGA(cm2/sec),PGV(cm/sec),SCORE
115,PEER,139,"Tabas, Iran",1978,7.35,Dayhook,471.53,13.940000,Reverse,0.000000,326.924291,25.872000,100.000000
63,AFAD,327966,17966,2023,7.70,Ali Tekden Sağlık Ocağı,424.00,38.529217,StrikeSlip,31.463333,286.722564,48.332219,91.304348
110,AFAD,327918,17966,2023,7.70,"Necip Fazıl Kültür Merkezi, Alparslan Türkeş B...",484.00,37.225843,StrikeSlip,43.853333,320.930444,37.353823,91.304348
61,AFAD,327952,17966,2023,7.70,Çok Programlı Anadolu Lisesi,448.00,20.737167,StrikeSlip,7.693333,291.290941,98.907393,91.304348
259,PEER,4451,"Montenegro, Yugo.",1979,7.10,Bar-Skupstina Opstine,462.23,6.980000,Reverse,0.000000,361.728092,49.588000,87.826087
366,PEER,8166,"Duzce, Turkey",1999,7.14,IRIGM 498,425.00,3.580000,StrikeSlip,0.000000,365.542879,25.269000,87.826087
260,PEER,4452,"Montenegro, Yugo.",1979,7.10,Debar - Skupstina Opstine,485.04,118.490000,Reverse,0.000000,55.347752,2.480000,69.565217
256,PEER,3821,Hector Mine,1999,7.13,San Pedro - Palos Verdes,477.92,206.310000,StrikeSlip,0.000000,8.361444,4.209900,69.565217
255,PEER,3798,Hector Mine,1999,7.13,Los Angeles - Colorado & Eagle Rock,418.83,177.390000,StrikeSlip,0.000000,20.845015,3.979400,69.565217
392,PEER,8605,El Mayor-Cucapah,2010,7.20,Bird Spring,402.00,388.790000,StrikeSlip,0.000000,2.152265,2.253900,69.565217


## FROM API FUNC

In [5]:
api = EarthquakeAPI(providers=[afadProvider,peerProvider], strategies=[strategy],search_criteria=search_criteria, target_params=target_params)

In [6]:
result = api.run_sync(criteria=search_criteria, target=target_params, strategy_name=strategy.get_name())

AFAD arama kriterleri: {'startDate': '2020-01-01T00:00:00.000Z', 'endDate': '2025-09-05T23:59:59.999Z', 'fromMagnitude': 7.0, 'toMagnitude': 10.0, 'fromVs30': 400, 'toVs30': 500}
AFAD'dan 114 kayıt alındı.
PEER'dan 279 kayıt alındı.


In [7]:
type(result)

selection_service.processing.ResultHandle.Result

In [8]:
type(result.value)

selection_service.core.Pipeline.PipelineResult

In [9]:
result.success

True

In [10]:
result.value.failed_providers

[]

In [11]:
result.value.execution_time

2.184022903442383

In [12]:
result.value.logs

['[OK] AFAD success',
 '[OK] PEER success',
 'Combined 2 datasets, total 393 records',
 'Strategy applied: TBDY_2018',
 'Execution time: 2.18 sec']

In [13]:
target_params

TargetParameters(magnitude=7.0, distance=30.0, vs30=400.0, mechanism=['StrikeSlip', 'Reverse'], pga=300, pgv=None, t90=None)

In [14]:
result.value.selected_df[['PROVIDER','RSN','EVENT','YEAR','MAGNITUDE','MAGNITUDE_TYPE','STATION','VS30(m/s)','RRUP(km)','MECHANISM','T90_avg(sec)','PGA(cm2/sec)','PGV(cm/sec)','SCORE']]

,PROVIDER,RSN,EVENT,YEAR,MAGNITUDE,MAGNITUDE_TYPE,STATION,VS30(m/s),RRUP(km),MECHANISM,T90_avg(sec),PGA(cm2/sec),PGV(cm/sec),SCORE
115,PEER,139,"Tabas, Iran",1978,7.35,Mw,Dayhook,471.53,13.940000,Reverse,0.000000,326.924291,25.872000,100.000000
110,AFAD,327918,17966,2023,7.70,MW,"Necip Fazıl Kültür Merkezi, Alparslan Türkeş B...",484.00,37.225843,StrikeSlip,43.853333,320.930444,37.353823,91.304348
65,AFAD,327966,17966,2023,7.70,MW,Ali Tekden Sağlık Ocağı,424.00,38.529217,StrikeSlip,31.463333,286.722564,48.332219,91.304348
64,AFAD,327952,17966,2023,7.70,MW,Çok Programlı Anadolu Lisesi,448.00,20.737167,StrikeSlip,7.693333,291.290941,98.907393,91.304348
259,PEER,4451,"Montenegro, Yugo.",1979,7.10,Mw,Bar-Skupstina Opstine,462.23,6.980000,Reverse,0.000000,361.728092,49.588000,87.826087
366,PEER,8166,"Duzce, Turkey",1999,7.14,Mw,IRIGM 498,425.00,3.580000,StrikeSlip,0.000000,365.542879,25.269000,87.826087
392,PEER,8605,El Mayor-Cucapah,2010,7.20,Mw,Bird Spring,402.00,388.790000,StrikeSlip,0.000000,2.152265,2.253900,69.565217
260,PEER,4452,"Montenegro, Yugo.",1979,7.10,Mw,Debar - Skupstina Opstine,485.04,118.490000,Reverse,0.000000,55.347752,2.480000,69.565217
256,PEER,3821,Hector Mine,1999,7.13,Mw,San Pedro - Palos Verdes,477.92,206.310000,StrikeSlip,0.000000,8.361444,4.209900,69.565217
255,PEER,3798,Hector Mine,1999,7.13,Mw,Los Angeles - Colorado & Eagle Rock,418.83,177.390000,StrikeSlip,0.000000,20.845015,3.979400,69.565217


In [15]:
result.value.scored_df[['PROVIDER','RSN','EVENT','YEAR','MAGNITUDE','STATION','VS30(m/s)','RRUP(km)','MECHANISM','T90_avg(sec)','PGA(cm2/sec)','PGV(cm/sec)','SCORE']]

,PROVIDER,RSN,EVENT,YEAR,MAGNITUDE,STATION,VS30(m/s),RRUP(km),MECHANISM,T90_avg(sec),PGA(cm2/sec),PGV(cm/sec),SCORE
0,AFAD,328381,17969,2023,7.6,İnönü Ortaokulu,444.0,88.355651,StrikeSlip,35.716667,42.899350,11.014204,60.869565
1,AFAD,328378,17969,2023,7.6,Meteoroloji Müdürlüğü,463.0,102.585602,StrikeSlip,47.253333,50.909903,21.016532,60.869565
2,AFAD,328386,17969,2023,7.6,Ortaokul,485.0,78.920309,StrikeSlip,36.536667,78.108384,13.673456,60.869565
3,AFAD,328392,17969,2023,7.6,Meteoroloji İstasyon Müdürlüğü,420.0,108.924054,StrikeSlip,18.980000,52.261725,7.239375,60.869565
4,AFAD,328432,17969,2023,7.6,,0.0,137.999196,StrikeSlip,66.973333,24.321866,10.951809,60.869565
...,...,...,...,...,...,...,...,...,...,...,...,...,...
388,PEER,8585,El Mayor-Cucapah,2010,7.2,Santa Barbara,485.0,417.450000,StrikeSlip,0.000000,1.808346,1.543500,69.565217
389,PEER,8587,El Mayor-Cucapah,2010,7.2,"San Clemente Island 2, Ca, Usa",442.0,256.830000,StrikeSlip,0.000000,3.907067,1.871500,69.565217
390,PEER,8592,El Mayor-Cucapah,2010,7.2,Slate Mountain,482.0,386.450000,StrikeSlip,0.000000,2.546297,1.134900,69.565217
391,PEER,8603,El Mayor-Cucapah,2010,7.2,Vincent Substation,405.0,298.860000,StrikeSlip,0.000000,4.136837,2.015600,69.565217


In [16]:
result.value.scored_df.sort_values(by=['SCORE'],ascending=True)[['PROVIDER','RSN','EVENT','YEAR','MAGNITUDE','STATION','VS30(m/s)','RRUP(km)','MECHANISM','T90_avg(sec)','PGA(cm2/sec)','PGV(cm/sec)','SCORE']]

,PROVIDER,RSN,EVENT,YEAR,MAGNITUDE,STATION,VS30(m/s),RRUP(km),MECHANISM,T90_avg(sec),PGA(cm2/sec),PGV(cm/sec),SCORE
264,PEER,4662,"Wenchuan, China",2008,7.90,Shawan,418.96,142.970000,Reverse/Oblique,0.000000,93.707444,7.757500,26.086957
283,PEER,4745,"Wenchuan, China",2008,7.90,Shimiancaoke,416.07,198.600000,Reverse/Oblique,0.000000,36.973032,3.364400,26.086957
281,PEER,4739,"Wenchuan, China",2008,7.90,Maerkangdizhenju,409.53,120.250000,Reverse/Oblique,0.000000,48.558608,2.861200,26.086957
280,PEER,4738,"Wenchuan, China",2008,7.90,Maerkangdiban,474.67,129.380000,Reverse/Oblique,0.000000,26.627997,2.528700,26.086957
279,PEER,4733,"Wenchuan, China",2008,7.90,Ludingshuichang,414.03,144.770000,Reverse/Oblique,0.000000,47.358274,2.959600,26.086957
...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,PEER,4451,"Montenegro, Yugo.",1979,7.10,Bar-Skupstina Opstine,462.23,6.980000,Reverse,0.000000,361.728092,49.588000,87.826087
64,AFAD,327952,17966,2023,7.70,Çok Programlı Anadolu Lisesi,448.00,20.737167,StrikeSlip,7.693333,291.290941,98.907393,91.304348
110,AFAD,327918,17966,2023,7.70,"Necip Fazıl Kültür Merkezi, Alparslan Türkeş B...",484.00,37.225843,StrikeSlip,43.853333,320.930444,37.353823,91.304348
65,AFAD,327966,17966,2023,7.70,Ali Tekden Sağlık Ocağı,424.00,38.529217,StrikeSlip,31.463333,286.722564,48.332219,91.304348


In [17]:
result.value.scored_df.describe().T

,count,mean,std,min,25%,50%,75%,max
RSN,393.0,97965.089059,147434.876447,13.000000,1573.000000,5847.000000,327993.000000,328703.000000
YEAR,393.0,2008.656489,11.135142,1952.000000,1999.000000,2010.000000,2023.000000,2023.000000
MAGNITUDE,393.0,7.497761,0.274283,7.000000,7.200000,7.600000,7.700000,7.900000
STATION_LAT,393.0,29.761681,18.186810,-46.250700,25.011700,34.140000,37.300000,65.311100
STATION_LON,393.0,23.138010,99.872726,-149.951200,-116.756000,36.305000,120.552000,174.915800
VS30(m/s),393.0,433.445318,81.953940,0.000000,420.000000,443.860000,469.650000,500.000000
STRIKE1,393.0,210.279898,119.050039,20.000000,85.100000,233.000000,330.000000,358.000000
DIP1,393.0,60.478372,19.394723,14.000000,50.000000,63.000000,74.000000,90.000000
RAKE1,393.0,58.816794,119.656468,-178.000000,18.000000,55.000000,174.000000,180.000000
RJB(km),393.0,184.028068,151.149207,0.000000,56.670000,146.510000,272.868152,713.780000


In [18]:
fdsn = providerFactory.create_provider(ProviderName.FDSN)
fdsn

In [19]:
search_criteria

SearchCriteria(start_date='2020-01-01', end_date='2025-09-05', min_magnitude=7.0, max_magnitude=10.0, min_depth=None, max_depth=None, station_code=None, network=None, country=None, province=None, district=None, neighborhood=None, min_latitude=None, max_latitude=None, min_longitude=None, max_longitude=None, circleLatitude=None, circleLongitude=None, circleRadius=None, min_pga=None, max_pga=None, min_pgv=None, max_pgv=None, min_pgd=None, max_pgd=None, fault_type=None, event_name=None, min_Repi=None, max_Repi=None, min_Rhyp=None, max_Rhyp=None, min_Rjb=None, max_Rjb=None, min_Rrup=None, max_Rrup=None, min_vs30=400, max_vs30=500, mechanisms=None, bbox=None, region=None)

In [20]:
criteria = fdsn.map_criteria(criteria=search_criteria)
criteria

{'starttime': 2020-01-01T00:00:00.000000Z,
 'endtime': 2025-09-05T00:00:00.000000Z,
 'minmagnitude': 7.0,
 'maxmagnitude': 10.0,
 'latitude': None,
 'longitude': None}

In [21]:
result = await fdsn.fetch_data_async(criteria=search_criteria)
result

<Result OK value=<class 'pandas.core.frame.DataFrame'>>

In [ ]:
async def main() -> Result[pd.DataFrame, ProviderError]:
    factory = ProviderFactory()
    fdsn = factory.create_provider(provider_type=ProviderName.FDSN)
    search_criteria = SearchCriteria(start_date="2020-01-01", end_date="2025-09-05", min_magnitude=7.0, max_magnitude=10.0, min_vs30=400, max_vs30=500 )
    result = await fdsn.fetch_data_async(criteria=search_criteria)
    return result
    
if __name__ == "__main__":
    test = asyncio.run(main())

In [ ]:
result.value

## DOWNLOAD AFAD WAVEFORMS

In [ ]:
result.value.selected_df.columns

In [ ]:
afad_filenames = result.value.selected_df[result.value.selected_df['PROVIDER'] == "AFAD"]['FILE_NAME_H1'].to_list()
afad_filenames

In [ ]:
response= afadProvider.download_afad_waveforms_batch(afad_filenames,file_status="RawAcc", batch_size=5, event_id="TEST_EVENT", user_name="GuestUser", export_type= "mseed")
response

In [ ]:
response.value

In [ ]:
# file_path="D:\\github\\SelectionEarthquake\\examples\\Afad_events\\TEST_EVENT\\0213\\20230206011732_0213.mseed"
file_path="D:\\github\\SelectionEarthquake\\examples\\Afad_events\\TEST_EVENT\\0213\\20230206011732_0213.mseed"

In [ ]:
# mseed_reader.py
import numpy as np
import re
from typing import Dict, List

class GroundMotionReader():
    """
    AFAD'ın metin tabanlı deprem veri formatını okuyan sınıf.
    """
    def __init__(self, file_path):
        self.file_path = file_path
        self.metadata = {}
        self.data = None
        self.units = "gal"
        self.fs = 100.0  # Varsayılan örnekleme frekansı (0.01 s → 100 Hz)

    def start(self):
        """Veri dosyasını okur ve verileri hazırlar."""
        self._parse_file()

    def _parse_file(self):
        """Dosyayı satır satır okuyarak metadata ve verileri ayırır."""
        with open(self.file_path, 'r', encoding='utf-8') as f:
            lines = f.readlines()
        
        metadata_lines = []
        data_lines = []
        in_data_section = False
        
        for line in lines:
            line = line.strip()
            if not line:
                continue
                
            if line.startswith(('N-S', 'E-W', 'U-D')) and not in_data_section:
                # Veri bölümü başlığı
                in_data_section = True
                continue
                
            if in_data_section:
                # Veri satırları
                if re.match(r'^-?\d+\.\d+', line):
                    data_lines.append(line)
            else:
                # Metadata satırları
                metadata_lines.append(line)
        
        # Metadata'ları parse et
        self._parse_metadata(metadata_lines)
        
        # Verileri parse et
        self._parse_data(data_lines)

    def _parse_metadata(self, lines: List[str]):
        """Metadata satırlarını parse eder."""
        for line in lines:
            if ':' in line:
                key, value = line.split(':', 1)
                key = key.strip()
                value = value.strip()
                self.metadata[key] = value
                
                # Önemli bilgileri ayıkla
                if 'SAMPLING INTERVAL' in key:
                    try:
                        interval = float(value.split()[0])
                        self.fs = 1.0 / interval  # Örnekleme frekansı
                    except:
                        pass
                elif 'NUMBER OF DATA' in key:
                    try:
                        self.n_data = int(value)
                    except:
                        pass
                elif 'RAW PGA VALUES' in key:
                    # PGA değerlerini ayıkla
                    pga_values = re.findall(r'\(([NSEWUD\-]+)\)\s+([\d\.]+)', value)
                    for direction, value in pga_values:
                        self.metadata[f'PGA_{direction}'] = float(value)

    def _parse_data(self, lines: List[str]):
        """Veri satırlarını parse eder."""
        ns_data = []
        ew_data = []
        ud_data = []
        
        for line in lines:
            # Satırdaki 3 float değerini ayıkla
            values = re.findall(r'-?\d+\.\d+', line)
            if len(values) == 3:
                ns_data.append(float(values[0]))
                ew_data.append(float(values[1]))
                ud_data.append(float(values[2]))
        
        # NumPy array'lerine dönüştür
        self.data = {
            'NS': np.array(ns_data),
            'EW': np.array(ew_data),
            'UD': np.array(ud_data)
        }
        
        # Zaman vektörünü oluştur
        n_samples = len(ns_data)
        self.time = np.arange(0, n_samples / self.fs, 1 / self.fs)

    def get_data(self) -> Dict:
        """Parselenen verileri döndürür."""
        if self.data is None:
            raise ValueError("Önce start() metodunu çağırın")
        
        return {
            "direction": ['NS', 'EW', 'UD'],
            "time": self.time,
            "acc": np.array([self.data['NS'], self.data['EW'], self.data['UD']]),
            "units": self.units,
            "fs": self.fs,
            "meta_data": self.metadata,
            "n_samples": len(self.time)
        }

    def stop(self):
        """Temizlik işlemleri."""
        pass

# Test

try:
    reader = GroundMotionReader(file_path)
    reader.start()
    data = reader.get_data()
    
    print("✓ Veri başarıyla okundu!")
    print(f"Örnek sayısı: {data['n_samples']}")
    print(f"Örnekleme frekansı: {data['fs']} Hz")
    print(f"Birimler: {data['units']}")
    print(f"Zaman aralığı: {data['time'][0]:.2f} - {data['time'][-1]:.2f} s")
    print(f"NS veri boyutu: {data['acc'][0].shape}")
    print(f"EW veri boyutu: {data['acc'][1].shape}")
    print(f"UD veri boyutu: {data['acc'][2].shape}")
    
    # İlk 5 örnek
    print("\nİlk 5 örnek:")
    for i in range(5):
        print(f"{data['time'][i]:6.3f}s: NS={data['acc'][0][i]:8.3f}, EW={data['acc'][1][i]:8.3f}, UD={data['acc'][2][i]:8.3f} {data['units']}")
    
    # Metadata
    print("\nÖnemli Metadata:")
    for key in ['PLACE', 'EARTHQUAKE DATE', 'EARTHQUAKE MAGNITUDE', 'STATION ID']:
        if key in data['meta_data']:
            print(f"{key}: {data['meta_data'][key]}")
            
except Exception as e:
    print(f"✗ Hata: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
import matplotlib.pyplot as plt
plt.plot(data['acc'][2])

In [ ]:
from obspy import UTCDateTime
from obspy.clients.fdsn import Client

# IRIS istemcisini oluştur
client = Client("IRIS")

# Deprem parametrelerini belirle
starttime = UTCDateTime("2024-01-01T00:00:00")
endtime = UTCDateTime("2025-01-02T00:00:00")
min_latitude = 35.0
max_latitude = 42.0
min_longitude = 25.0
max_longitude = 45.0
min_magnitude = 4.0

# Deprem kataloğunu çek
catalog = client.get_events(
    starttime=starttime,
    endtime=endtime,
    minlatitude=min_latitude,
    maxlatitude=max_latitude,
    minlongitude=min_longitude,
    maxlongitude=max_longitude,
    minmagnitude=min_magnitude
)

print(f"Bulunan deprem sayısı: {len(catalog)}")

# Depremleri listele
for i, event in enumerate(catalog):
    origin = event.preferred_origin()
    magnitude = event.preferred_magnitude()
    print(f"{i+1}. Deprem:")
    print(f"   Zaman: {origin.time}")
    print(f"   Enlem: {origin.latitude:.2f}")
    print(f"   Boylam: {origin.longitude:.2f}")
    print(f"   Derinlik: {origin.depth/1000:.1f} km")
    print(f"   Büyüklük: {magnitude.mag}")
    print("-" * 50)

In [ ]:
dir(catalog[0])

In [ ]:
type(catalog[0])

In [ ]:
catalog[0].__dict__

In [ ]:
from obspy import UTCDateTime
from obspy.clients.fdsn import Client
import matplotlib.pyplot as plt

def deprem_dalga_formu_cek(deprem_zamani, istasyon, sure=300):
    """Belirli bir depremin dalga formunu çek"""
    client = Client("IRIS")
    
    # Depremden sonraki dalga formlarını çek
    start = deprem_zamani - 30  # Depremden 30 saniye önce
    end = start + sure  # Toplam süre
    
    try:
        # Dalga formu verisini çek
        stream = client.get_waveforms(
            network="IU",  # Global istasyon ağı
            station=istasyon,
            location="00",
            channel="BHZ",  # Düşey bileşen
            starttime=start,
            endtime=end
        )
        
        # Veriyi işle
        stream.detrend('linear')
        stream.taper(max_percentage=0.05)
        stream.filter('bandpass', freqmin=0.01, freqmax=1.0)
        
        # Grafik çiz
        plt.figure(figsize=(12, 6))
        for i, trace in enumerate(stream):
            plt.plot(trace.times(), trace.data, label=trace.id)
        
        plt.axvline(x=30, color='red', linestyle='--', label='Deprem Zamanı')
        plt.xlabel('Zaman (s)')
        plt.ylabel('Genlik')
        plt.title(f'Deprem Dalga Formu - İstasyon: {istasyon}')
        plt.legend()
        plt.grid(True)
        plt.show()
        
        return stream
        
    except Exception as e:
        print(f"Hata: {e}")
        return None

# Örnek kullanım
deprem_zamani = UTCDateTime("2024-01-01T10:30:00")
deprem_dalga_formu_cek(deprem_zamani,None, 300)